In [1]:
import tensorflow as tf
from tensorflow import keras
from keras import layers
from keras.datasets import cifar10, mnist
from keras.utils import to_categorical

tf.random.set_seed(42)

In [2]:
# Загрузка данных
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

# Нормализация
X_train = X_train / 255.
X_test = X_test / 255.

# One_hot_encoding
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [3]:
print(X_train[0], print(y_train[0]))
print(X_train.shape, y_train.shape)

[0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
[[[0.23137255 0.24313725 0.24705882]
  [0.16862745 0.18039216 0.17647059]
  [0.19607843 0.18823529 0.16862745]
  ...
  [0.61960784 0.51764706 0.42352941]
  [0.59607843 0.49019608 0.4       ]
  [0.58039216 0.48627451 0.40392157]]

 [[0.0627451  0.07843137 0.07843137]
  [0.         0.         0.        ]
  [0.07058824 0.03137255 0.        ]
  ...
  [0.48235294 0.34509804 0.21568627]
  [0.46666667 0.3254902  0.19607843]
  [0.47843137 0.34117647 0.22352941]]

 [[0.09803922 0.09411765 0.08235294]
  [0.0627451  0.02745098 0.        ]
  [0.19215686 0.10588235 0.03137255]
  ...
  [0.4627451  0.32941176 0.19607843]
  [0.47058824 0.32941176 0.19607843]
  [0.42745098 0.28627451 0.16470588]]

 ...

 [[0.81568627 0.66666667 0.37647059]
  [0.78823529 0.6        0.13333333]
  [0.77647059 0.63137255 0.10196078]
  ...
  [0.62745098 0.52156863 0.2745098 ]
  [0.21960784 0.12156863 0.02745098]
  [0.20784314 0.13333333 0.07843137]]

 [[0.70588235 0.54509804 0.37647059]
  [0.

In [4]:
inputs = keras.Input(shape = (32, 32, 3))
x = layers.Conv2D(32, 3, activation='relu')(inputs) # padding = 'valid' is default
x = layers.Conv2D(64, 3, activation='relu')(x)
block_1_output = layers.MaxPooling2D(3)(x)
x = layers.Conv2D(64, 3, activation='relu', padding='same')(block_1_output)
x = layers.Conv2D(64, 3, activation='relu', padding='same')(x)
block_2_output = layers.add([x, block_1_output])
x = layers.Conv2D(64, 3, activation='relu', padding='same')(block_2_output)
x = layers.Conv2D(64, 3, activation='relu', padding='same')(x)
block_3_output = layers.add([x, block_2_output])
x = layers.Conv2D(64, 3, activation='relu')(block_3_output)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(256, activation='relu')(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(10, activation = 'softmax')(x)

In [5]:
model = keras.Model(inputs, outputs, name = 'toy_resnet')
model.summary()

Model: "toy_resnet"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 conv2d (Conv2D)                (None, 30, 30, 32)   896         ['input_1[0][0]']                
                                                                                                  
 conv2d_1 (Conv2D)              (None, 28, 28, 64)   18496       ['conv2d[0][0]']                 
                                                                                                  
 max_pooling2d (MaxPooling2D)   (None, 9, 9, 64)     0           ['conv2d_1[0][0]']               
                                                                                         

In [8]:
model.compile(optimizer = 'adam',
              loss = 'categorical_crossentropy',
              metrics = ['accuracy'])
    
model.fit(X_train, y_train, batch_size = 64, epochs = 15, validation_split=0.2)

Epoch 1/15
625/625 [==============================] - 136s 212ms/step - loss: 1.8041 - accuracy: 0.2975 - val_loss: 1.4523 - val_accuracy: 0.4593
Epoch 2/15
625/625 [==============================] - 129s 207ms/step - loss: 1.3628 - accuracy: 0.4968 - val_loss: 1.1764 - val_accuracy: 0.5788
Epoch 3/15
625/625 [==============================] - 132s 211ms/step - loss: 1.1390 - accuracy: 0.5879 - val_loss: 1.0244 - val_accuracy: 0.6262
Epoch 4/15
625/625 [==============================] - 164s 263ms/step - loss: 0.9806 - accuracy: 0.6488 - val_loss: 0.8876 - val_accuracy: 0.6825
Epoch 5/15
625/625 [==============================] - 160s 256ms/step - loss: 0.8858 - accuracy: 0.6855 - val_loss: 0.8417 - val_accuracy: 0.7023
Epoch 6/15
625/625 [==============================] - 143s 228ms/step - loss: 0.8086 - accuracy: 0.7125 - val_loss: 0.7998 - val_accuracy: 0.7208
Epoch 7/15
625/625 [==============================] - 128s 205ms/step - loss: 0.7437 - accuracy: 0.7365 - val_loss: 0.7670 -

In [9]:
model.evaluate(X_test, y_test)

313/313 [==============================] - 10s 31ms/step - loss: 0.7366 - accuracy: 0.7693


[0.7366470098495483, 0.7692999839782715]